In [24]:
import random
import numpy as np
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
Image.MAX_IMAGE_PIXELS = None

# LOADING

In [76]:
### Load Province information
prov = {}
with open("input files/abw_provinces.txt","r") as file:
    contents = file.read().splitlines()

for line in range(0, len(contents)-1, 12):
    ids = int(contents[line].split("=")[0].split('\t')[-1])
    prov[ids] = {
        'loc': [None]*2,
        'terrain': contents[line+1].split('"')[1],
        'culture': contents[line+2].split('"')[1],
        'religion': contents[line+3].split('"')[1],
        'trade_goods': contents[line+4].split('"')[1],
        'civilization_value': int(contents[line+5].split('=')[1]),
        'barbarian_power': int(contents[line+6].split('=')[1]),
        'province_rank': contents[line+7].split('"')[1],
        'pops': contents[line+8],
        'holysite': contents[line+9]
    }

### Load countries from setup
f = open("input files/setup_countriesonly.txt","r")
contents = f.read().splitlines()
f.close()
country = {}
for line in range(2, len(contents)-4, 9):
    tag = contents[line].split(' ')[0].split('\t')[2]
    temp = contents[line+1].split('#')[1].split(',')
    country[tag] = {
        'rgb': tuple([ int(x) for x in temp ]),
        'government': contents[line+2],
        'culture': contents[line+3].split('=')[1].split(' ')[1],
        'religion': contents[line+4].split('=')[1].split(' ')[1],
        'regnal': contents[line+5],
        'capital': contents[line+6],
        'owned': contents[line+7].split('{ ')[1].split(' }')[0].split(' ')
    }


### Load areas
with open("input files/areas.txt","r") as file:
    contents = file.read().splitlines()

areas = {}
temp = len(contents)
for line in range(0, len(contents)-3, 4):
    name = contents[line].split(' =')[0]
    areas[name] = list(map(int,contents[line+1].split('= { ')[1].split(' }')[0].split(' ')))

### Load regions
with open("input files/regions.txt","r") as file:
    contents = file.read().splitlines()

regions = {}
temp = len(contents)
for line in range(0, len(contents)-3, 4):
    name = contents[line].split(' =')[0]
    regions[name] = contents[line+1].split('= { ')[1].split(' }')[0].split(' ')

### Load localization
with open("input files/provincenames_l_english.yml","r") as file:
    contents = file.read().splitlines()

provNames = {}
for line in range(1, len(contents)):
    provNames[line] = contents[line].split('"')[1]

In [110]:
### Load locators
locators = {
    'combat_locators': {},
    'city_locators': {},
    'fort_locators': {},
    'great_work_locators': {},
    'unit_stack_locators': {},
    'vfx_locators': {},
    'port_locators': {}
}
headers = {}

for key in locators:
    with open(f'input files/locators/{key}.txt', 'r') as file:
        contents = file.read().splitlines()
        headers[key] = contents[0:7]
        for line in range(8, len(contents)-4, 6):
            ids = int(contents[line].split("=")[1])
            locators[key][ids] = contents[line+1:line+4]


In [42]:
## Load CSVs
definitions = pd.read_csv('input files/definition.csv', delimiter=';')
display(definitions.head(5))
ports = pd.read_csv('input files/ports.csv', delimiter=';')
display(ports.head(5))

,#id,R,G,B,Name,x,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,","
0,0,0,0,0,NaN,x,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,","
1,1,0,0,0,Aegmacia,x,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,","
2,2,171,128,0,Chalcleslia,x,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,","
3,3,0,128,0,Erytherylia,x,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,","
4,4,115,0,0,Locrlesiana,x,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,","


,LandProvince,SeaZone,x,y,Unnamed: 4
0,2,9598,4407,4877,NaN
1,3,9940,9164,4523,NaN
2,4,9587,14424,4459,NaN
3,5,9396,14397,4426,NaN
4,6,9689,14257,4420,NaN


In [77]:
## Load province IDs to delete
with open("input files/delete_provs.txt","r") as file:
    contents = file.read().splitlines()

deleteprovs = []
for line in contents:
    deleteprovs.append(int(line.split(' ')[0]))

# COMPUTE

In [172]:
imgProvs = np.array(Image.open('input files/provinces.png')).reshape(-1,3)
imgProvs = np.unique(imgProvs, axis=0)

In [178]:
imgProvs.shape

(5642, 3)

In [78]:
i=1
for ids in prov:
    if ids in deleteprovs:
        prov[ids]['nextId'] = None
    else:
        prov[ids]['nextId'] = i
        i+=1

In [79]:
for area in areas:
    temp = []
    areas[area] = [x for x in areas[area] if x not in deleteprovs]
    

In [161]:
for name in country:
    temp = country[name]['capital'].split(' = ')[1].split('\t')
    temp[0] = str(prov[int(temp[0])]['nextId'])
    
    if len(temp)==2: country[name]['capital'] = f'\t\t\tcapital = {temp[0]}\t{temp[1]}'
    else: country[name]['capital'] = f'\t\t\tcapital = {temp[0]}'

    temp = []
    for ids in country[name]['owned']:
        nextId = prov[int(ids)]['nextId']
        if nextId==None: continue
        else:
            temp.append(str(nextId))
    
    country[name]['owned'] = temp

# SAVING

In [131]:
with open('output files/regions.txt', 'w') as file:
    for reg in regions:
        file.write(reg+' = { #'+str(len(regions[reg]))+'\n\tareas = { ')
        for area in regions[reg]: file.write(area+' ')
        file.write('}\n}\n\n')

with open('output files/areas.txt', 'w') as file:
    for area in areas:
        file.write(area+' = { #'+str(len(areas[area]))+'\n\tprovinces = { ')
        for ids in areas[area]: file.write(str(prov[ids]['nextId'])+' ')
        file.write('}\n}\n\n')

In [118]:
def write_provinces(filename, prov):
    with open('output files/'+filename+'.txt','w') as file:
        for ids in prov:
            if prov[ids]['nextId']==None: continue
            file.write('\t'+str(prov[ids]['nextId'])+'={\n')
            file.write('\t\tterrain="'+prov[ids]['terrain']+'"\n')
            file.write('\t\tculture="'+prov[ids]['culture']+'"\n')
            file.write('\t\treligion="'+prov[ids]['religion']+'"\n')
            file.write('\t\ttrade_goods="'+prov[ids]['trade_goods']+'"\n')
            file.write('\t\tcivilization_value='+str(prov[ids]['civilization_value'])+'\n')
            file.write('\t\tbarbarian_power='+str(prov[ids]['barbarian_power'])+'\n')
            file.write('\t\tprovince_rank="'+prov[ids]['province_rank']+'"\n')
            file.write(prov[ids]['pops']+'\n')
            file.write(prov[ids]['holysite']+'\n')
            file.write('\t}\n\n')

In [119]:
### Write the province setup as multiple files (I:Rome standard)
temp = {}
for ids in prov: # First the file that holds every inhabitable province
    if prov[ids]['pops'][-1] == '#': temp[ids] = prov[ids].copy() #In my case this is what I use to tell if a province is habitable
write_provinces('by_region/00_default', temp)

for r_name in regions: # Now by region
    temp = {}
    templist = []
    for a_name in regions[r_name]: 
        
        try: templist.extend(areas[a_name])
        except: print(a_name)
    templist.sort()
    for ids in templist: temp[ids] = prov[ids].copy()
    write_provinces('by_region/00_'+r_name, temp)

In [120]:
### Write the province setup as 1 file
write_provinces('abw_provinces', prov)

In [121]:
### Write CSVs
with open('output files/definition.csv', 'w') as file:
    file.write('#id;R;G;B;Name;x;;;;;;;;;;;;;;;;;,\n')
    file.write('0;0;0;0;;x;;;;;;;;;;;;;;;;;,\n')
    for ids in prov:
        nextId = prov[ids]['nextId']
        if nextId==None: continue
        else: 
            color = definitions.iloc[ids,1:4].to_list()
            name = definitions.iloc[ids,4]
            file.write(f'{nextId};{color[0]};{color[1]};{color[2]};{name};x;;;;;;;;;;;;;;;;;,\n')

####
with open('output files/ports.csv', 'w') as file:
    file.write('LandProvince;SeaZone;x;y;\n')
    for i in range(len(ports)-1):
        prevId = ports.iloc[i,0]
        prevSeaId = ports.iloc[i,1]
        x = ports.iloc[i,2]
        y = ports.iloc[i,3]
        
        try: nextId = prov[prevId]['nextId']
        except: print(prevId)
        if nextId==None: continue
        else:
            nextSeaId = prov[prevSeaId]['nextId']
            file.write(f'{nextId};{nextSeaId};{x};{y};\n')
    file.write('-1;-1;-1;-1;')

In [122]:
### Write localization
with open('output files/provincenames_l_english.yml', 'w') as file:
    file.write('l_english:\n')
    for ids in prov:
        nextId = prov[ids]['nextId']
        if nextId==None: continue
        else: 
            name = provNames[ids]
            file.write(f' PROV{nextId}:0 "{name}"\n')

In [123]:
### Write locators
for key in locators:
    with open(f'output files/locators/{key}.txt', 'w') as file:
        for elem in headers[key]:
            file.write(elem+'\n')
        for ids in locators[key]:
            if ids==0: nextId = 0
            else:
                nextId = prov[ids]['nextId']
                if nextId==None: continue
            file.write('\t\t{\n\t\t\tid='+str(nextId)+'\n')
            for elem in locators[key][ids]:
                file.write(elem+'\n')
            file.write('\t\t}\n')

In [164]:
### Write the country setup
with open('output files/setup_countriesonly.txt','w') as f:
    f.write('country = {\n\tcountries = {')
    for tag in country:
        f.write('\n\t\t'+tag+' = {')
        f.write('\n\t\t\t#'+str(country[tag]['rgb'][0])+','+str(country[tag]['rgb'][1])+','+str(country[tag]['rgb'][2]))
        f.write('\n'+country[tag]['government'])
        f.write('\n\t\t\tprimary_culture = '+country[tag]['culture'])
        f.write('\n\t\t\treligion = '+country[tag]['religion'])
        f.write('\n'+country[tag]['regnal']+'\n'+country[tag]['capital'])
        f.write('\n\t\t\town_control_core = 	{ ')
        for ids in country[tag]['owned']: f.write(ids+' ')
        f.write('}\n\t\t}')
    f.write('\n\t}\n}')